[CSV file dumps](https://api.covid19india.org/csv)

Approach:

1. Download all the three csv files: [raw_data1.csv](https://api.covid19india.org/csv/latest/raw_data1.csv), [raw_data2.csv](https://api.covid19india.org/csv/latest/raw_data2.csv) and [raw_data3.csv](https://api.covid19india.org/csv/latest/raw_data3.csv)

2. Concatenate `raw_data1.csv` & `raw_data2.csv` >> `raw_data` 

3. Add column `Num Cases' = `1` to the above file

4. Add dummy columns `Estimated Onset Date` & `Backup Notes` to `raw_data3.csv`

5. For `raw_data3.csv` keep only rows with `Current Status` == `Hospitalized`

6. Concatenate `raw_data3.csv` with `raw_data`

7. Save the file to csv. Filename has datestamp. 

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import json

In [2]:
# Get csv file & json (latest) dumps
!wget https://api.covid19india.org/csv/latest/raw_data1.csv
!wget https://api.covid19india.org/csv/latest/raw_data2.csv
!wget https://api.covid19india.org/csv/latest/raw_data3.csv

--2020-05-02 13:50:27--  https://api.covid19india.org/csv/latest/raw_data1.csv
Resolving api.covid19india.org (api.covid19india.org)... 185.199.109.153, 185.199.108.153, 185.199.110.153, ...
Connecting to api.covid19india.org (api.covid19india.org)|185.199.109.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2835015 (2.7M) [text/csv]
Saving to: ‘raw_data1.csv’

raw_data1.csv       100%[===================>]   2.70M  --.-KB/s    in 0.03s   

2020-05-02 13:50:28 (94.3 MB/s) - ‘raw_data1.csv’ saved [2835015/2835015]

--2020-05-02 13:50:28--  https://api.covid19india.org/csv/latest/raw_data2.csv
Resolving api.covid19india.org (api.covid19india.org)... 185.199.111.153, 185.199.110.153, 185.199.109.153, ...
Connecting to api.covid19india.org (api.covid19india.org)|185.199.111.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1538031 (1.5M) [text/csv]
Saving to: ‘raw_data2.csv’

raw_data2.csv       100%[===================>]   1.47M  --

In [3]:
raw_data1 = pd.read_csv('raw_data1.csv')
raw_data2 = pd.read_csv('raw_data2.csv')
raw_data3 = pd.read_csv('raw_data3.csv')

In [4]:
print(raw_data1.columns)
print(raw_data2.columns)
print(raw_data3.columns)

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Estimated Onset Date', 'Age Bracket', 'Gender', 'Detected City',
       'Detected District', 'Detected State', 'State code', 'Current Status',
       'Notes', 'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')
Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Estimated Onset Date', 'Age Bracket', 'Gender', 'Detected City',
       'Detected District', 'Detected State', 'State code', 'Current Status',
       'Notes', 'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')
Index(['Entry_ID', 'State Patient Number', 'Date Announced', 'Age Bracket',
       'Gender', 'Detected City', 'Detected District', 'Detected S

In [5]:
raw_data1['Current Status'].value_counts()

Hospitalized    17079
Recovered         181
Deceased           45
Migrated            1
Name: Current Status, dtype: int64

> raw_data3 -> 'Estimated Onset Date', 'Backup Notes'

> raw_data1 & 2 -> 'Num Cases' = 1

In [6]:
raw_data3 = raw_data3.rename(columns ={'Entry_ID':'Patient Number'}) #Rename column name identifying patient
raw_data3['Patient Number'] = raw_data3['Patient Number'].apply(lambda x: str(x) + "_27APR") 

raw_data3 = raw_data3[raw_data3['Current Status'] == 'Hospitalized']

sum(raw_data3['Date Announced'].isnull()) # Not required since the above step removes these rows 
raw_data3.dropna(subset=['Date Announced'])

#raw_data3['Date Announced'] = pd.to_datetime(raw_data3['Date Announced'], format = "%d/%m/%Y")
raw_data3['Num Cases'] = raw_data3['Num Cases'].astype('float64')

In [7]:
min(raw_data3['Date Announced'])

'01/05/2020'

In [8]:
raw_data = pd.concat([raw_data1, raw_data2])
print(raw_data.shape)
raw_data.head()

(27891, 20)


,Patient Number,State Patient Number,Date Announced,Estimated Onset Date,Age Bracket,Gender,Detected City,Detected District,Detected State,State code,Current Status,Notes,Contracted from which Patient (Suspected),Nationality,Type of transmission,Status Change Date,Source_1,Source_2,Source_3,Backup Notes
0,1,KL-TS-P1,30/01/2020,NaN,20,F,Thrissur,Thrissur,Kerala,KL,Recovered,Travelled from Wuhan,NaN,India,Imported,14/02/2020,https://twitter.com/vijayanpinarayi/status/122...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan
1,2,KL-AL-P1,02/02/2020,NaN,NaN,NaN,Alappuzha,Alappuzha,Kerala,KL,Recovered,Travelled from Wuhan,NaN,India,Imported,14/02/2020,https://www.indiatoday.in/india/story/kerala-r...,https://weather.com/en-IN/india/news/news/2020...,NaN,Student from Wuhan
2,3,KL-KS-P1,03/02/2020,NaN,NaN,NaN,Kasaragod,Kasaragod,Kerala,KL,Recovered,Travelled from Wuhan,NaN,India,Imported,14/02/2020,https://www.indiatoday.in/india/story/kerala-n...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan
3,4,DL-P1,02/03/2020,NaN,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,DL,Recovered,"Travelled from Austria, Italy",NaN,India,Imported,15/03/2020,https://www.indiatoday.in/india/story/not-a-ja...,https://economictimes.indiatimes.com/news/poli...,NaN,Travel history to Italy and Austria
4,5,TS-P1,02/03/2020,NaN,24,M,Hyderabad,Hyderabad,Telangana,TG,Recovered,"Travelled from Dubai to Bangalore on 20th Feb,...",NaN,India,Imported,02/03/2020,https://www.deccanherald.com/national/south/qu...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact"


In [9]:
# Adding missing columns in raw_data & raw_data3 dataframes
raw_data['Num Cases'] = 1

raw_data3['Estimated Onset Date'] = 0 
raw_data3['Backup Notes'] = 0

In [10]:
raw_data3 = raw_data3[raw_data3['Current Status'] == 'Hospitalized']

In [11]:
raw_data = pd.concat([raw_data, raw_data3])

In [12]:
raw_data['Age Bracket'] = raw_data['Age Bracket'].apply(lambda x : 30 if str(x).__contains__('-') else x)# Around 4-5 rows have age ranges 28-35. Replacing it with 30
raw_data['Date Announced'] = pd.to_datetime(raw_data['Date Announced'], format = "%d/%m/%Y")

raw_data['Age Bracket'] = raw_data['Age Bracket'].astype('float64')
raw_data['Gender'] = raw_data['Gender'].astype('category')
raw_data['Detected City'] = raw_data['Detected City'].astype('category')
raw_data['Detected District'] = raw_data['Detected District'].astype('category')
raw_data['Detected State'] = raw_data['Detected State'].astype('category')

raw_data['Nationality'] = raw_data['Nationality'].astype('category')
raw_data['Type of transmission'] = raw_data['Type of transmission'].astype('category')

In [13]:
print(min(raw_data['Date Announced']))
print(max(raw_data['Date Announced']))

2020-01-30 00:00:00
2020-05-02 00:00:00


In [14]:
from datetime import date

currdate = str(date.today())
#date.time()

In [15]:
raw_data.to_csv(f'raw_data_{currdate}.csv')

In [16]:
!ls

__notebook__.ipynb  raw_data2.csv  raw_data_2020-05-02.csv
raw_data1.csv	    raw_data3.csv
